<a href="https://colab.research.google.com/github/rohithreddy999/Machine_learning-AI-predictions/blob/main/predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/AUTOMATED_WEATHER_CLASSIFICATION_USING_TRANSFER_LEARNING/archive.zip"

Archive:  /content/drive/MyDrive/AUTOMATED_WEATHER_CLASSIFICATION_USING_TRANSFER_LEARNING/archive.zip
  inflating: Multi-class Weather Dataset/Cloudy/cloudy1.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy10.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy100.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy101.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy102.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy103.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy104.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy105.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy106.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy107.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy108.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy109.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy11.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy110.jpg  
  infl

In [3]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [4]:
base_dir="/content/Multi-class Weather Dataset"

In [5]:
train_gen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train = train_gen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size= 32,
    class_mode='categorical',
    subset='training')

validation_gen = train_gen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size= 32,
    class_mode='categorical',
    subset='validation')

Found 901 images belonging to 4 classes.
Found 224 images belonging to 4 classes.


In [6]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [7]:
# Adding the preprocessing layer to the front of vgg
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
# Train model with existing weights
for layer in vgg.layers:
  print(layer)

In [9]:
# Train model with existing weights
for layer in vgg.layers:
  layer.trainable=False

In [10]:
x = Flatten()(vgg.output)

In [11]:
# output layer
prediction = Dense(4,activation='softmax')(x)
# Create Vgg16 model
model = Model(inputs=vgg.input,outputs=prediction)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [14]:
model.fit_generator(train,validation_data=validation_gen,epochs=5,steps_per_epoch=len(train),
                    validation_steps=len(validation_gen))

<ipython-input-14-d078b837adfc>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=validation_gen,epochs=5,steps_per_epoch=len(train),


Epoch 1/5
29/29 [==============================] - 36s 788ms/step - loss: 0.8106 - accuracy: 0.6792 - val_loss: 0.5073 - val_accuracy: 0.8036
Epoch 2/5
29/29 [==============================] - 20s 678ms/step - loss: 0.2468 - accuracy: 0.9279 - val_loss: 0.3871 - val_accuracy: 0.8705
Epoch 3/5
29/29 [==============================] - 21s 713ms/step - loss: 0.1950 - accuracy: 0.9412 - val_loss: 0.3475 - val_accuracy: 0.8795
Epoch 4/5
29/29 [==============================] - 21s 738ms/step - loss: 0.1430 - accuracy: 0.9623 - val_loss: 0.5057 - val_accuracy: 0.8304
Epoch 5/5
29/29 [==============================] - 20s 679ms/step - loss: 0.1045 - accuracy: 0.9745 - val_loss: 0.3571 - val_accuracy: 0.8795


In [62]:
model.save("weather_prediction.h5")

In [15]:
#RESNET 50
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [16]:
for layer in resnet.layers:
  layer.trainable=False

In [17]:
for layer in resnet.layers:
  print(layer)

In [18]:
x = Flatten()(resnet.output)

In [19]:
out = Dense(4, activation='softmax')(x)

In [20]:
res_model = Model(inputs=resnet.input,outputs=out)

In [21]:
res_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [22]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
res_model.fit(train,epochs=5,validation_data=validation_gen,steps_per_epoch=len(train),
              validation_steps=len(validation_gen))

Epoch 1/5
29/29 [==============================] - 26s 757ms/step - loss: 7.0846 - accuracy: 0.2797 - val_loss: 2.1847 - val_accuracy: 0.2589
Epoch 2/5
29/29 [==============================] - 20s 678ms/step - loss: 2.2538 - accuracy: 0.3596 - val_loss: 1.6889 - val_accuracy: 0.3616
Epoch 3/5
29/29 [==============================] - 20s 711ms/step - loss: 1.4062 - accuracy: 0.4306 - val_loss: 0.9950 - val_accuracy: 0.5670
Epoch 4/5
29/29 [==============================] - 20s 681ms/step - loss: 1.0172 - accuracy: 0.5738 - val_loss: 1.3676 - val_accuracy: 0.4911
Epoch 5/5
29/29 [==============================] - 20s 710ms/step - loss: 1.0231 - accuracy: 0.5483 - val_loss: 0.8117 - val_accuracy: 0.6786


In [65]:
#inseption-v3
from tensorflow.keras.applications.inception_v3 import InceptionV3
Inception = InceptionV3(include_top=False,input_shape=(150, 150, 3))

In [66]:
for layer in resnet.layers:
  layer.trainable=False

In [67]:
for layer in resnet.layers:
  print(layer)

In [68]:
x1= Flatten()(Inception.output)

In [69]:
output = Dense(4, activation='softmax')(x1)

In [70]:
inception_model = Model(inputs=Inception.input,outputs=output)

In [71]:
inception_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 74, 74, 32)   864         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [76]:
inception_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])